# Wall Chebyshev Ground State Projector Circuits in Guppy

In [1]:
from wallcheb.qtmlib.circuits.lcu import LCUMultiplexorBox
from guppylang import guppy
from guppylang.std.builtins import comptime
from guppylang.std.quantum import qubit, measure_array, h,x
from guppylang.std.builtins import array, exit
from typing import Callable
from guppylang.std.debug import state_result


- Prepare each shift Hamiltonian in the product:

$$
    g^{\text{wall-Ch}}_m(\hat H) = \prod_{\nu=1}^m \frac{\hat H-a_{\nu}}{S-a_{\nu}}.
$$

Here $S$ is an estimate of the unkown ground state energy $E_0$ and $a_\nu$ is defined in Eq. 16 in the paper. In this example $\hat{H}$ is the Hubbard Hamiltonian:

$$
    \hat{H} = -t \sum_{\langle i,j \rangle} (c_i^\dagger c_j + c_j^\dagger c_i) + U \sum_i n_{i\uparrow} n_{i\downarrow}
$$

where we set $t=1$.

In [2]:
from wallcheb.operators import generate_pytket_hvs_hubbard

u = 1
n_sites = 2
m = 4
product_block_encoding_qpo, hubbard_hamiltonian = generate_pytket_hvs_hubbard(u, n_sites, m)
n_state_qubits = 2*n_sites


/Users/nathanfitzpatric/Github/wall_chebyshev/wallcheb/operators/hubbard_model.py:54: ComplexWarning: Casting complex values to real discards the imaginary part
  matrix[i,j] = hmat[subset[i], subset[j]]


- For benchmarking the circuit implementation we obtain the exact ground state and its energy using exact diagonalization.

In [3]:
from numpy.linalg import eigh

e,c = eigh(hubbard_hamiltonian)
c[:,0], e[0]

(matrix([[-0.00000000e+00-0.j],
         [-0.00000000e+00-0.j],
         [-2.22044605e-16+0.j],
         [-4.35162146e-01+0.j],
         [ 0.00000000e+00+0.j],
         [ 0.00000000e+00+0.j],
         [ 5.57345410e-01+0.j],
         [ 0.00000000e+00+0.j],
         [-9.49975516e-17+0.j],
         [-5.57345410e-01+0.j],
         [ 0.00000000e+00+0.j],
         [ 0.00000000e+00+0.j],
         [-4.35162146e-01+0.j],
         [-0.00000000e+00+0.j],
         [-0.00000000e+00+0.j],
         [-0.00000000e+00-0.j]]),
 np.float64(-1.5615528128088303))

- We use the block encoding method from the [paper](https://arxiv.org/abs/2501.18515) called the Multiplexor LCU.
- We intialise the first term to ger the number of prepare qubits for the LCUs.

In [4]:
lcu_box = LCUMultiplexorBox(product_block_encoding_qpo[0], n_state_qubits)
n_prep_qubits = lcu_box.n_prepare_qubits
n_qubits = n_prep_qubits + n_state_qubits

- The block encoding was previously implemented in pytket, so we import it into guppy.

In [5]:
from pytket.passes import AutoRebase
from pytket import OpType
from pytket.passes import DecomposeBoxes


def build_multiplexor_lcu(ham, n_state_qubits, ind):
    multiplexor_lcu = LCUMultiplexorBox(ham, n_state_qubits)
    

    circ = multiplexor_lcu.get_circuit()
    DecomposeBoxes().apply(circ)
    rebase = AutoRebase({OpType.CX, OpType.Rz, OpType.H, OpType.CCX})
    rebase.apply(circ)


    qlibs_multiplexor_lcu = guppy.load_pytket(f"qlibs_multiplexor_lcu_{ind}", circ)
    return qlibs_multiplexor_lcu

- To build the guppy circuits we need to run it at compile time and import into the main function.

In [6]:
@guppy.comptime
def guppy_prod_circs() -> array[Callable[[array[qubit, comptime(n_prep_qubits)], array[qubit, comptime(n_state_qubits)]],None], comptime(m)]:

    guppy_circuits = [build_multiplexor_lcu(qpo, n_state_qubits, i) for i, qpo in enumerate(product_block_encoding_qpo)]
    return guppy_circuits

- The the individual LCU terms in product are provided as a list of guppy functions. 
- This are then appended together we the state register and the prepare register is measured and post selected.

In [7]:
from guppylang.std.builtins import comptime
from guppylang.std.quantum import qubit, discard_array, measure, measure_array
from hugr.qsystem.result import QsysResult
from guppylang.std.builtins import result, array, exit
from typing import Callable

@guppy
def product_block_encoding(prod_block_encoding: array[Callable[[array[qubit, comptime(n_prep_qubits)], array[qubit, comptime(n_state_qubits)]], None], comptime(m)], state_qreg: array[qubit, comptime(n_state_qubits)]) -> None:
    

    for prod_block in prod_block_encoding.copy():

        prep_qreg = array(qubit() for _ in range(comptime(n_prep_qubits)))
        prod_block(prep_qreg, state_qreg)

        outcome = measure_array(prep_qreg)
        # result("c", measure_array(prep_qubits))
        for b in outcome: 
            if b:
                exit("circuit failed",1)




- We start from the Hartree-Fock state and then apply the products of block encodings
- We then measure the final statevector
- Note: This is only for testing - on a real quantum computer you cannot access the statevector directly.

In [8]:
@guppy
def main() -> None:
    """Main function to run the multiplexor LCU circuit."""
    state_qreg = array(qubit() for _ in range(comptime(n_state_qubits)))
    
    x(state_qreg[0])  # Set the first qubit to |1> state
    x(state_qreg[3])

    prods = guppy_prod_circs()

    product_block_encoding(prods, state_qreg)

    state_result("gs", state_qreg)
    # result('c',measure_array(state_qreg))
    discard_array(state_qreg)

compiled_hugr = guppy.compile(main)


- We then run the circuit with the get_state_vector method
- Note: The for larger $m$ the success probability may be so low that no shots make it the end of the circuit and an will result in an error.

In [9]:
from wallcheb.utils import get_state_vector, compute_expectation

sv = get_state_vector(compiled_hugr, n_qubits, n_shots=25000)
expectation_value = compute_expectation(sv, hubbard_hamiltonian)
    

- The resulting projected energy:

In [10]:
expectation_value

matrix([[-1.52304659+0.j]])

- Error is the expectation value of the Hamiltonian in the projected state.

In [11]:
e[0] - expectation_value

matrix([[-0.03850622+0.j]])

In [ ]:
data_by_category['rz_count']

[]

# Gate Counts
- The scaling of the Multiplexor block encoding is linear in the number of applicatons of the Hamiltonian.

In [ ]:
import numpy as np
circ = lcu_box.get_circuit()
DecomposeBoxes().apply(circ)
rebase = AutoRebase({OpType.CX, OpType.Rz, OpType.H, OpType.CCX})
rebase.apply(circ)

circ.n_2qb_gates()
circ.n_gates
circ.depth()

data = np.array([circ.n_2qb_gates(), circ.n_gates, circ.depth()])

m=10
m_prod_data = [np.array([i+1])*data.copy() for i in range(m)]


#make into a dataframe
import pandas as pd
df = pd.DataFrame(m_prod_data, columns=['n_2qb_gates', 'n_gates', 'depth'])
df['m'] = np.arange(m)
df.set_index('m', inplace=True)
df    

[array([166, 615, 266]), array([ 332, 1230,  532]), array([ 498, 1845,  798]), array([ 664, 2460, 1064]), array([ 830, 3075, 1330]), array([ 996, 3690, 1596]), array([1162, 4305, 1862]), array([1328, 4920, 2128]), array([1494, 5535, 2394]), array([1660, 6150, 2660])]


,n_2qb_gates,n_gates,depth
m,,,
0,166,615,266
1,332,1230,532
2,498,1845,798
3,664,2460,1064
4,830,3075,1330
5,996,3690,1596
6,1162,4305,1862
7,1328,4920,2128
8,1494,5535,2394
